In [1]:
import os

In [2]:
%pwd

'd:\\University Documents\\Projects\\Text-Summarizer\\research'

In [3]:
os.chdir('../')
print(f"New working directory: {os.getcwd()}")

New working directory: d:\University Documents\Projects\Text-Summarizer


In [4]:
print(f"Current working directory: {os.getcwd()}")

Current working directory: d:\University Documents\Projects\Text-Summarizer


In [5]:
# creating entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    # return types of the function
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

[2024-07-26 19:33:57,076: INFO: __init__: Welcome to our custom logging]
Log file path: d:\University Documents\Projects\Text-Summarizer\logs\running_logs.log


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root ])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:             # DataIngestionConfig is the return type of the method
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
# Data Ingestion components
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    # download the data from the url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """ zip_file_path: str
            extracts the zip file into the data directory
            Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# updating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file() 
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-26 19:37:33,610: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-26 19:37:33,610: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-26 19:37:33,618: INFO: common: created directory at: artifacts]
[2024-07-26 19:37:33,618: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-26 19:37:39,534: INFO: 1956493964: artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 24844620
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "013ed835e3bb3f5a031bc5d9f9bfd837301884e12550e607b54d582d49e8b329"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D964:212CDD:5035FE:5CA76A:66A3ADA8
Accept-Ranges: bytes
Date: Fri, 26 Jul 2024 14:07:38 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4720-BOM
X-Ca